<a href="https://colab.research.google.com/github/Maoko81/test/blob/main/newGrowthAccounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np


# Load data
pwt101 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt101.loc[pwt101['country'].isin(["France", "Germany", "Canada", "Italy", "Japan", "United Kingdom", "United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional columns
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker
data['a'] = 1 - data['labsh']  

# Order by year
data = data.sort_values('year')

# Group by countrycode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  
data['k'] = (grouped_data['k_pc'].diff() * 100)  
data['tfp'] = data['g'] - data['a'] * data['k']  

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean',
                                            'tfp': 'mean',
                                               'k': "mean",
                                               'a': "mean"})

# Calculate additional summary statistics
summary['tfp.growth'] = summary['tfp']
summary['capitakl deepning'] = summary['a']*summary['k']
summary['tfp.share'] = summary['tfp'] / summary['g']
summary['capital.share'] = summary['a']
summary['growth rate'] = summary['g']

# Print output
print(summary)

                    g       tfp         k         a  tfp.growth  \
countrycode                                                       
CAN          0.887700  0.253687  1.835712  0.346190    0.253687   
DEU          0.661055  0.223337  1.198290  0.370730    0.223337   
FRA          0.829236  0.260418  1.491602  0.381194    0.260418   
GBR          1.015544  0.442543  1.379846  0.412993    0.442543   
ITA         -0.037181 -0.571016  1.103655  0.484835   -0.571016   
JPN          0.710956  0.079903  1.506205  0.430601    0.079903   
USA          1.536985  0.756660  2.014710  0.393260    0.756660   

             capitakl deepning  tfp.share  capital.share  growth rate  
countrycode                                                            
CAN                   0.635505   0.285780       0.346190     0.887700  
DEU                   0.444242   0.337849       0.370730     0.661055  
FRA                   0.568590   0.314046       0.381194     0.829236  
GBR                   0.569867   0.4